In [17]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from lyricsgenius import Genius

In [32]:
col_names=['pid', 'track_uri', 'position', 'artist_uri', 'album_uri', 'track_name',
       'artist_name', 'album_name', 'duration']
data=pd.read_csv('interactions_with_track_name.csv',nrows=70000)
data.head()

,pid,track_uri,position,artist_uri,album_uri,track_name,artist_name,album_name,duration
0,0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0,spotify:artist:2wIVse2owClT7go1WT98tk,spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,Lose Control (feat. Ciara & Fat Man Scoop),Missy Elliott,The Cookbook,226863
1,0,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,1,spotify:artist:26dSoYclwsYLMAKD3tpOr4,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,Toxic,Britney Spears,In The Zone,198800
2,0,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,2,spotify:artist:6vWDO969PvNqNYHIOW5v0m,spotify:album:25hVFAxTlDvXbx2X2QkUkE,Crazy In Love,Beyoncé,Dangerously In Love (Alben für die Ewigkeit),235933
3,0,spotify:track:1AWQoqb9bSvzTjaLralEkT,3,spotify:artist:31TPClRtHm23RisEBtV3X7,spotify:album:6QPkyl04rXwTGlGlcYaRoW,Rock Your Body,Justin Timberlake,Justified,267266
4,0,spotify:track:1lzr43nnXAijIGYnCT8M8H,4,spotify:artist:5EvFsr3kj42KNv97ZEnqij,spotify:album:6NmFmPX56pcLBOFMhIiKvF,It Wasn't Me,Shaggy,Hot Shot,227600


In [33]:
tracks=data[['track_uri','artist_name','track_name']].values.tolist()

In [34]:
token='PFl5Jdd01ayEMNqxIkuoAWnA7N9Xw9KqD9BSphLmjQ4IBrJqyaTA9CxKP2k8yJpz'
genius = Genius(token)
genius.timeout=300

In [35]:
genius.verbose=False

In [113]:
lyric_col=[]
tryagain=[tracks[2890]]
art,titles,uris=[],[],[]
for uri, artist, title in tqdm(tracks[2890+4707+1208+8910+5391+2092+7929+917+2364+506:]):
    try:
        song = genius.search_song(title, artist)
        if song==None:
#             print(f"Song not found - {r['Title']} by {r['Artist']}")
            tryagain.append([uri,artist,title])
            continue
        lyrics=song.lyrics
        if lyrics.count('-')>200:
#             print(f"Song not found - {r['Title']} by {r['Artist']}")
            tryagain.append([uri,artist,title])
            continue
        verses=[]
        for x in lyrics.split('Lyrics')[1][:-7].split('\n'):
            if '[' in list(x) or len(x)==0:
                continue
            verses.append(x.replace("\'",""))
        lyric_col.append(' '.join(verses))
        art.append(artist)
        titles.append(title)
        uris.append(uri)
    except TimeoutError:
        tryagain.append([uri,artist,title])
    

100%|██████████████████████████████████| 33086/33086 [16:14:36<00:00,  1.77s/it]


In [152]:
newdf=pd.DataFrame()
newdf['artist']=art
newdf['title']=titles
newdf['uri']=uris
newdf['lyrics']=lyric_col

In [155]:
newdf

,artist,title,uri,lyrics
0,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,"Music make you lose control, music make you lo..."
1,Britney Spears,Toxic,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,"Baby, cant you see Im calling? A guy like you ..."
2,Beyoncé,Crazy In Love,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,"Yes! (Whoo, ow!) So crazy right now Most incre..."
3,Justin Timberlake,Rock Your Body,spotify:track:1AWQoqb9bSvzTjaLralEkT,Dont be so quick to walk away Dance with me I ...
4,Shaggy,It Wasn't Me,spotify:track:1lzr43nnXAijIGYnCT8M8H,"Yo, man Yo Open up, man Yo, what you want, man..."
...,...,...,...,...
58826,Greenleaf,Stray Bullit Woman,spotify:track:6U7yjgpweQna55cfWEZXIk,"Oscar: Arise, warrior. The battle is about to ..."
58827,Danzig,Twist Of Cain,spotify:track:25ZH79eefRHT8CUo15UFjo,"Yea, yea, yea Ooh I can feel it move me Feel i..."
58828,Danzig,On A Wicked Night,spotify:track:6DsjI4sPDFSLf6t0SYlbxi,"Hey, my lady in black Now I see youre back Hav..."
58829,Eagles,Those Shoes,spotify:track:13TrgZ4RqtjXd8w9wAuwxE,Tell us what youre gonna do tonight to mama Th...


In [115]:
# newdf.to_csv('spotify-mpd-lyrics.csv',index=False)

In [116]:
# pd.DataFrame(tryagain,columns=['uri','artist','track']).to_csv('no-lyrics.csv',index=False)